In [1]:
import numpy as np
import scipy
import numba
import matplotlib
import seaborn

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Numba version:", numba.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", seaborn.__version__)


NumPy version: 1.24.4
SciPy version: 1.13.1
Numba version: 0.60.0
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [1]:
import pandas as pd
import numpy as np
import wrangle as w
from datetime import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
df = w.clean_data('nfl_data.csv')
df

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,1,6,6,3,127,7,13,2,2,32:16
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,2,2,3,134,9,16,0,0,35:53
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,3,3,3,6,292,4,16,1,2,36:16
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,3,3,2,76,4,9,0,1,38:13


In [4]:
df.shape

(12350, 63)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12350 entries, 0 to 12349
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                12350 non-null  int64  
 1   team                  12350 non-null  object 
 2   week                  12350 non-null  int64  
 3   day                   12350 non-null  object 
 4   date                  12350 non-null  object 
 5   result                12350 non-null  object 
 6   ot                    12350 non-null  object 
 7   host                  12350 non-null  object 
 8   opp                   12350 non-null  object 
 9   pts                   12350 non-null  int64  
 10  opp_pts               12350 non-null  int64  
 11  pass_cmp              12350 non-null  int64  
 12  pass_att              12350 non-null  int64  
 13  pass_yds              12350 non-null  int64  
 14  pass_td               12350 non-null  int64  
 15  int                

In [6]:
df.iloc[:16]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
5,2000,CRD,7,Sun,October 15,L,no,home,Philadelphia Eagles,14,...,2,3,4,3,42,10,18,2,2,39:34
6,2000,CRD,8,Sun,October 22,L,no,away,Dallas Cowboys,7,...,2,6,6,3,38,3,7,0,0,30:45
7,2000,CRD,9,Sun,October 29,L,no,home,New Orleans Saints,10,...,1,3,3,5,118,6,14,0,0,26:41
8,2000,CRD,10,Sun,November 5,W,no,home,Washington Redskins,16,...,5,0,0,0,0,8,15,0,1,37:26
9,2000,CRD,11,Sun,November 12,L,no,away,Minnesota Vikings,14,...,1,4,4,1,39,6,9,0,1,34:08


In [7]:
df[(df['season'] == 2000) & (df['team'].str.contains('nyg', case=False))]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
352,2000,NYG,1,Sun,September 3,W,no,home,Arizona Cardinals,21,...,1,1,1,5,105,6,15,0,2,28:30
353,2000,NYG,2,Sun,September 10,W,no,away,Philadelphia Eagles,33,...,2,0,1,4,82,3,10,0,1,21:16
354,2000,NYG,3,Sun,September 17,W,no,away,Chicago Bears,14,...,1,1,1,7,211,3,12,0,1,21:49
355,2000,NYG,4,Sun,September 24,L,no,home,Washington Redskins,6,...,2,1,2,5,88,5,14,0,0,31:22
356,2000,NYG,5,Sun,October 1,L,no,away,Tennessee Titans,14,...,1,4,4,4,66,14,20,0,1,42:46
357,2000,NYG,6,Sun,October 8,W,no,away,Atlanta Falcons,13,...,3,0,0,5,92,5,14,0,1,30:15
358,2000,NYG,7,Sun,October 15,W,no,home,Dallas Cowboys,19,...,0,2,2,3,39,5,13,0,1,30:42
359,2000,NYG,9,Sun,October 29,W,no,home,Philadelphia Eagles,24,...,1,1,1,9,300,1,11,0,1,16:19
360,2000,NYG,10,Sun,November 5,W,no,away,Cleveland Browns,24,...,1,0,0,6,176,5,13,0,1,23:20
361,2000,NYG,11,Sun,November 12,L,no,home,St. Louis Rams,24,...,2,5,5,6,144,3,13,1,1,36:40


In [38]:
df[df['team'].str.contains('NYG')]

,season,team,week,day,date,result,ot,host,opp,pts,...,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id
352,2000,NYG,1,Sun,September 3,W,no,home,Arizona Cardinals,21,...,1,1,5,105,6,15,0,2,28:30,2000_1_Arizona Cardinals_NYG
353,2000,NYG,2,Sun,September 10,W,no,away,Philadelphia Eagles,33,...,0,1,4,82,3,10,0,1,21:16,2000_2_NYG_Philadelphia Eagles
354,2000,NYG,3,Sun,September 17,W,no,away,Chicago Bears,14,...,1,1,7,211,3,12,0,1,21:49,2000_3_Chicago Bears_NYG
355,2000,NYG,4,Sun,September 24,L,no,home,Washington Redskins,6,...,1,2,5,88,5,14,0,0,31:22,2000_4_NYG_Washington Redskins
356,2000,NYG,5,Sun,October 1,L,no,away,Tennessee Titans,14,...,4,4,4,66,14,20,0,1,42:46,2000_5_NYG_Tennessee Titans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12209,2023,NYG,14,Mon,December 11,W,no,home,Green Bay Packers,24,...,1,1,3,119,5,14,1,1,31:06,2023_14_Green Bay Packers_NYG
12210,2023,NYG,15,Sun,December 17,L,no,away,New Orleans Saints,6,...,3,3,4,167,6,12,0,0,33:27,2023_15_NYG_New Orleans Saints
12211,2023,NYG,16,Mon,December 25,L,no,away,Philadelphia Eagles,25,...,3,3,2,108,8,15,2,2,34:38,2023_16_NYG_Philadelphia Eagles
12212,2023,NYG,17,Sun,December 31,L,no,home,Los Angeles Rams,25,...,2,4,5,287,2,8,0,1,30:11,2023_17_Los Angeles Rams_NYG


In [40]:
df['team'].unique().tolist()

['CRD',
 'ATL',
 'RAV',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GNB',
 'CLT',
 'JAX',
 'KAN',
 'SDG',
 'RAM',
 'RAI',
 'MIA',
 'MIN',
 'NWE',
 'NOR',
 'NYG',
 'NYJ',
 'PHI',
 'PIT',
 'SEA',
 'SFO',
 'TAM',
 'OTI',
 'WAS',
 'HTX']

In [41]:
# The mapping of team abbreviations to full names
team_code_map = {
    'CRD': 'Arizona Cardinals',
    'ATL': 'Atlanta Falcons',
    'RAV': 'Baltimore Ravens',
    'BUF': 'Buffalo Bills',
    'CAR': 'Carolina Panthers',
    'CHI': 'Chicago Bears',
    'CIN': 'Cincinnati Bengals',
    'CLE': 'Cleveland Browns',
    'DAL': 'Dallas Cowboys',
    'DEN': 'Denver Broncos',
    'DET': 'Detroit Lions',
    'GNB': 'Green Bay Packers',
    'CLT': 'Indianapolis Colts',
    'JAX': 'Jacksonville Jaguars',
    'KAN': 'Kansas City Chiefs',
    'SDG': 'Los Angeles Chargers',
    'RAM': 'Los Angeles Rams',
    'RAI': 'Las Vegas Raiders',
    'MIA': 'Miami Dolphins',
    'MIN': 'Minnesota Vikings',
    'NWE': 'New England Patriots',
    'NOR': 'New Orleans Saints',
    'NYG': 'New York Giants',
    'NYJ': 'New York Jets',
    'PHI': 'Philadelphia Eagles',
    'PIT': 'Pittsburgh Steelers',
    'SEA': 'Seattle Seahawks',
    'SFO': 'San Francisco 49ers',
    'TAM': 'Tampa Bay Buccaneers',
    'OTI': 'Tennessee Titans',
    'WAS': 'Washington Commanders',
    'HTX': 'Houston Texans'
}

# Step 1: Replace team abbreviations in the 'team' column with full names
df['team_full_name'] = df['team'].map(team_code_map)

In [91]:
df['name_x_season']= df['opp']

In [94]:
df['opp'].sort_values().unique().tolist()

['Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Titans',
 'Washington Commanders',
 'Washington Football Team',
 'Washington Redskins']

In [96]:
# Original teams
teams_original = [
    'Arizona Cardinals',
    'Atlanta Falcons',
    'Baltimore Ravens',
    'Buffalo Bills',
    'Carolina Panthers',
    'Chicago Bears',
    'Cincinnati Bengals',
    'Cleveland Browns',
    'Dallas Cowboys',
    'Denver Broncos',
    'Detroit Lions',
    'Green Bay Packers',
    'Houston Texans',
    'Indianapolis Colts',
    'Jacksonville Jaguars',
    'Kansas City Chiefs',
    'Las Vegas Raiders',
    'Los Angeles Chargers',
    'Los Angeles Rams',
    'Miami Dolphins',
    'Minnesota Vikings',
    'New England Patriots',
    'New Orleans Saints',
    'New York Giants',
    'New York Jets',
    'Oakland Raiders',  # Will be replaced
    'Philadelphia Eagles',
    'Pittsburgh Steelers',
    'San Diego Chargers',  # Will be replaced
    'San Francisco 49ers',
    'Seattle Seahawks',
    'St. Louis Rams',  # Will be replaced
    'Tampa Bay Buccaneers',
    'Tennessee Titans',
    'Washington Commanders',
    'Washington Football Team',  # Will be replaced
    'Washington Redskins'  # Will be replaced
]

# Mapping
team_mapping = {
    'Oakland Raiders': 'Las Vegas Raiders',
    'San Diego Chargers': 'Los Angeles Chargers',
    'St. Louis Rams': 'Los Angeles Rams',
    'Washington Football Team': 'Washington Commanders',
    'Washington Redskins': 'Washington Commanders'
}

# Update the original list to reflect current teams
updated_teams = [team_mapping.get(team, team) for team in teams_original]

# Remove duplicates
updated_teams = list(dict.fromkeys(updated_teams))

In [96]:
# Original teams
teams_original = [
    'Arizona Cardinals',
    'Atlanta Falcons',
    'Baltimore Ravens',
    'Buffalo Bills',
    'Carolina Panthers',
    'Chicago Bears',
    'Cincinnati Bengals',
    'Cleveland Browns',
    'Dallas Cowboys',
    'Denver Broncos',
    'Detroit Lions',
    'Green Bay Packers',
    'Houston Texans',
    'Indianapolis Colts',
    'Jacksonville Jaguars',
    'Kansas City Chiefs',
    'Las Vegas Raiders',
    'Los Angeles Chargers',
    'Los Angeles Rams',
    'Miami Dolphins',
    'Minnesota Vikings',
    'New England Patriots',
    'New Orleans Saints',
    'New York Giants',
    'New York Jets',
    'Oakland Raiders',  # Will be replaced
    'Philadelphia Eagles',
    'Pittsburgh Steelers',
    'San Diego Chargers',  # Will be replaced
    'San Francisco 49ers',
    'Seattle Seahawks',
    'St. Louis Rams',  # Will be replaced
    'Tampa Bay Buccaneers',
    'Tennessee Titans',
    'Washington Commanders',
    'Washington Football Team',  # Will be replaced
    'Washington Redskins'  # Will be replaced
]

# Mapping
team_mapping = {
    'Oakland Raiders': 'Las Vegas Raiders',
    'San Diego Chargers': 'Los Angeles Chargers',
    'St. Louis Rams': 'Los Angeles Rams',
    'Washington Football Team': 'Washington Commanders',
    'Washington Redskins': 'Washington Commanders'
}

# Update the original list to reflect current teams
updated_teams = [team_mapping.get(team, team) for team in teams_original]

# Remove duplicates
updated_teams = list(dict.fromkeys(updated_teams))

In [39]:
team_mapping = {
    'Arizona Cardinals': 'Arizona Cardinals',
    'Atlanta Falcons': 'Atlanta Falcons',
    'Baltimore Ravens': 'Baltimore Ravens',
    'Buffalo Bills': 'Buffalo Bills',
    'Carolina Panthers': 'Carolina Panthers',
    'Chicago Bears': 'Chicago Bears',
    'Cincinnati Bengals': 'Cincinnati Bengals',
    'Cleveland Browns': 'Cleveland Browns',
    'Dallas Cowboys': 'Dallas Cowboys',
    'Denver Broncos': 'Denver Broncos',
    'Detroit Lions': 'Detroit Lions',
    'Green Bay Packers': 'Green Bay Packers',
    'Houston Texans': 'Houston Texans',
    'Indianapolis Colts': 'Indianapolis Colts',
    'Jacksonville Jaguars': 'Jacksonville Jaguars',
    'Kansas City Chiefs': 'Kansas City Chiefs',
    'Las Vegas Raiders': 'Las Vegas Raiders',  # Updated
    'Los Angeles Chargers': 'Los Angeles Chargers',  # Updated
    'Los Angeles Rams': 'Los Angeles Rams',  # Updated
    'Miami Dolphins': 'Miami Dolphins',
    'Minnesota Vikings': 'Minnesota Vikings',
    'New England Patriots': 'New England Patriots',
    'New Orleans Saints': 'New Orleans Saints',
    'New York Giants': 'New York Giants',
    'New York Jets': 'New York Jets',
    'Oakland Raiders': 'Las Vegas Raiders',  # Will be replaced
    'Philadelphia Eagles': 'Philadelphia Eagles',
    'Pittsburgh Steelers': 'Pittsburgh Steelers',
    'San Diego Chargers': 'Los Angeles Chargers',  # Will be replaced
    'San Francisco 49ers': 'San Francisco 49ers',
    'Seattle Seahawks': 'Seattle Seahawks',
    'St. Louis Rams': 'Los Angeles Rams',  # Will be replaced
    'Tampa Bay Buccaneers': 'Tampa Bay Buccaneers',
    'Tennessee Titans': 'Tennessee Titans',
    'Washington Commanders': 'Washington Commanders',  # Updated
    'Washington Football Team': 'Washington Commanders',  # Will be replaced
    'Washington Redskins': 'Washington Commanders'  # Will be replaced
}

In [108]:
df['opp'] = df['opp'].map(team_mapping)

In [111]:
df

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,3,127,7,13,2,2,32:16,2023_13_Miami Dolphins_Washington Commanders,Washington Commanders,Miami Dolphins
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,134,9,16,0,0,35:53,2023_15_Los Angeles Rams_Washington Commanders,Washington Commanders,Los Angeles Rams
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,6,292,4,16,1,2,36:16,2023_16_New York Jets_Washington Commanders,Washington Commanders,New York Jets
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,76,4,9,0,1,38:13,2023_17_San Francisco 49ers_Washington Commanders,Washington Commanders,San Francisco 49ers


In [112]:
#Create a unique identifier for each game
# This ensures each game is represented only once, even if it's recorded from both teams' perspectives
df['game_id'] = df.apply(lambda row: f"{row['season']}_{row['week']}_{'_'.join(sorted([row['team_full_name'], row['opp']]))}", axis=1)

In [113]:
df

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,3,127,7,13,2,2,32:16,2023_13_Miami Dolphins_Washington Commanders,Washington Commanders,Miami Dolphins
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,134,9,16,0,0,35:53,2023_15_Los Angeles Rams_Washington Commanders,Washington Commanders,Los Angeles Rams
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,6,292,4,16,1,2,36:16,2023_16_New York Jets_Washington Commanders,Washington Commanders,New York Jets
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,76,4,9,0,1,38:13,2023_17_San Francisco 49ers_Washington Commanders,Washington Commanders,San Francisco 49ers


In [114]:
# Step 2: Drop duplicate entries based on the game_id
# Keep only one entry for each game (e.g., keeping the first occurrence)
df_cleaned = df.drop_duplicates(subset='game_id')

# Step 3: Drop the 'game_id' column if it's no longer needed
#df_cleaned = df_cleaned.drop(columns=['game_id'])

In [115]:
df_cleaned

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12277,2023,SEA,14,Sun,December 10,L,no,away,San Francisco 49ers,16,...,5,246,6,11,0,0,29:15,2023_14_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers
12279,2023,SEA,16,Sun,December 24,W,no,away,Tennessee Titans,20,...,3,117,6,12,1,1,33:51,2023_16_Seattle Seahawks_Tennessee Titans,Seattle Seahawks,Tennessee Titans
12291,2023,SFO,11,Sun,November 19,W,no,home,Tampa Bay Buccaneers,27,...,3,144,7,14,1,3,29:09,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers
12297,2023,SFO,17,Sun,December 31,W,no,away,Washington Commanders,27,...,3,156,3,9,0,1,21:47,2023_17_San Francisco 49ers_Washington Commanders,San Francisco 49ers,Washington Commanders


In [116]:
df.shape

(12350, 66)

In [117]:
df_cleaned.shape

(6175, 66)

In [118]:
12350/2

6175.0

In [119]:
df_cleaned.sort_values(by = ['season' , 'week']).head(18)

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
16,2000,ATL,1,Sun,September 3,W,no,home,San Francisco 49ers,36,...,5,98,2,9,0,1,28:21,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers
32,2000,RAV,1,Sun,September 3,W,no,away,Pittsburgh Steelers,16,...,6,184,4,14,0,2,24:53,2000_1_Baltimore Ravens_Pittsburgh Steelers,Baltimore Ravens,Pittsburgh Steelers
48,2000,BUF,1,Sun,September 3,W,no,home,Tennessee Titans,16,...,9,308,2,14,0,1,26:49,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans
64,2000,CAR,1,Sun,September 3,L,no,away,Washington Commanders,17,...,4,64,5,12,0,1,33:14,2000_1_Carolina Panthers_Washington Commanders,Carolina Panthers,Washington Redskins
80,2000,CHI,1,Sun,September 3,L,no,away,Minnesota Vikings,27,...,2,156,6,13,0,0,27:26,2000_1_Chicago Bears_Minnesota Vikings,Chicago Bears,Minnesota Vikings
112,2000,CLE,1,Sun,September 3,L,no,home,Jacksonville Jaguars,7,...,3,35,10,16,0,0,37:03,2000_1_Cleveland Browns_Jacksonville Jaguars,Cleveland Browns,Jacksonville Jaguars
128,2000,DAL,1,Sun,September 3,L,no,home,Philadelphia Eagles,14,...,2,176,7,12,0,0,39:30,2000_1_Dallas Cowboys_Philadelphia Eagles,Dallas Cowboys,Philadelphia Eagles
144,2000,DEN,1,Mon,September 4,L,no,away,Los Angeles Rams,36,...,1,34,6,7,0,0,27:47,2000_1_Denver Broncos_Los Angeles Rams,Denver Broncos,St. Louis Rams
160,2000,DET,1,Sun,September 3,W,no,away,New Orleans Saints,14,...,7,240,6,17,0,1,29:11,2000_1_Detroit Lions_New Orleans Saints,Detroit Lions,New Orleans Saints


In [74]:
pd.options.display.max_rows = 20

In [122]:
df_cleaned.sort_values(by = ['season' , 'week'])

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
16,2000,ATL,1,Sun,September 3,W,no,home,San Francisco 49ers,36,...,5,98,2,9,0,1,28:21,2000_1_Atlanta Falcons_San Francisco 49ers,Atlanta Falcons,San Francisco 49ers
32,2000,RAV,1,Sun,September 3,W,no,away,Pittsburgh Steelers,16,...,6,184,4,14,0,2,24:53,2000_1_Baltimore Ravens_Pittsburgh Steelers,Baltimore Ravens,Pittsburgh Steelers
48,2000,BUF,1,Sun,September 3,W,no,home,Tennessee Titans,16,...,9,308,2,14,0,1,26:49,2000_1_Buffalo Bills_Tennessee Titans,Buffalo Bills,Tennessee Titans
64,2000,CAR,1,Sun,September 3,L,no,away,Washington Commanders,17,...,4,64,5,12,0,1,33:14,2000_1_Carolina Panthers_Washington Commanders,Carolina Panthers,Washington Redskins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,2023,JAX,18,Sun,January 7,L,no,away,Tennessee Titans,20,...,3,144,6,11,0,1,27:43,2023_18_Jacksonville Jaguars_Tennessee Titans,Jacksonville Jaguars,Tennessee Titans
12077,2023,KAN,18,Sun,January 7,W,no,away,Los Angeles Chargers,13,...,5,218,7,19,2,3,32:55,2023_18_Kansas City Chiefs_Los Angeles Chargers,Kansas City Chiefs,Los Angeles Chargers
12111,2023,RAM,18,Sun,January 7,W,no,away,San Francisco 49ers,21,...,3,144,5,10,1,1,29:00,2023_18_Los Angeles Rams_San Francisco 49ers,Los Angeles Rams,San Francisco 49ers
12179,2023,NWE,18,Sun,January 7,L,no,home,New York Jets,3,...,7,322,3,15,0,1,33:42,2023_18_New England Patriots_New York Jets,New England Patriots,New York Jets


In [123]:
# Step 4: Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('cleaned_nfl_dataset.csv', index=False)


In [3]:
df = pd.read_csv('cleaned_nfl_dataset.csv')

In [15]:
df.columns[:9]

Index(['season', 'team', 'week', 'day', 'date', 'result', 'ot', 'host', 'opp'], dtype='object')

In [8]:
df['opp'].sort_values().unique().tolist()

['Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'Tampa Bay Buccaneers',
 'Tennessee Titans',
 'Washington Commanders']

In [19]:
df[df['opp'].str.contains('c', case=False)].tail()

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
6168,2023,SEA,10,Sun,November 12,W,no,home,Washington Commanders,29,...,5,227,7,15,1,1,29:33,2023_10_Seattle Seahawks_Washington Commanders,Seattle Seahawks,Washington Commanders
6169,2023,SEA,12,Thu,November 23,L,no,home,San Francisco 49ers,13,...,5,199,5,11,0,0,35:30,2023_12_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers
6170,2023,SEA,14,Sun,December 10,L,no,away,San Francisco 49ers,16,...,5,246,6,11,0,0,29:15,2023_14_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers
6172,2023,SFO,11,Sun,November 19,W,no,home,Tampa Bay Buccaneers,27,...,3,144,7,14,1,3,29:09,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers
6173,2023,SFO,17,Sun,December 31,W,no,away,Washington Commanders,27,...,3,156,3,9,0,1,21:47,2023_17_San Francisco 49ers_Washington Commanders,San Francisco 49ers,Washington Commanders


In [18]:
df['team_full_name'].sort_values().unique().tolist()

['Arizona Cardinals',
 'Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'Tampa Bay Buccaneers',
 'Tennessee Titans']

In [20]:
df['team'].nunique()

31

In [37]:
pd.options.display.max_columns = 20

In [23]:
df.head()

,season,team,week,day,date,result,ot,host,opp,pts,opp_pts,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,name_x_season
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,21,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,28:30,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,31,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,31:20,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,29,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,23:32,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,27,24,41,239,1,0,0,0,5.8,5.8,58.5,83.3,24,126,5.3,1,2,3,2,2,4,176,3,13,2,3,27:50,22,33,215,1,0,1,5,6.7,6.3,66.7,94.9,32,130,4.1,2,2,2,3,3,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,21,17,30,169,2,0,1,2,5.7,5.5,56.7,95.0,31,146,4.7,1,3,3,2,2,3,144,2,10,1,2,31:03,16,22,136,0,0,1,2,6.3,5.9,72.7,88.4,31,104,3.4,3,0,1,3,3,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns


In [25]:
df.rename(columns={'name_x_season': 'opp_full_name'}, inplace=  True)

In [39]:
df.to_csv('cleaned_nfl_dataset.csv', index= False)

In [42]:
df.head()

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns


In [ ]:
# We can use logistic regression to make a prediction (a probability between 0 and 1) of the home team winning or losing. 
# Create a Home_team / away_team columns -- done
# encode categorical columns
# change time / date format on T.O.P columns and dates -- done

In [2]:
df = pd.read_csv('cleaned_nfl_dataset.csv')

In [4]:
df.head()

,season,team,week,day,date,result,ot,host,opp,pts,...,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns


In [11]:
# Made home and away columns 
df['home_team'] = df.apply(lambda row: row['team_full_name'] if row['host'] == 'home' else row['opp_full_name'], axis = 1)
df['away_team'] = df.apply(lambda row: row['opp_full_name'] if row['host'] == 'home' else row['team_full_name'], axis = 1)
df.head()

,season,team,week,day,date,result,ot,host,opp,pts,...,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,Arizona Cardinals
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns,Arizona Cardinals,Cleveland Browns


In [15]:
# Function to convert "minutes:seconds" to total seconds
def convert_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# Apply the function to the 'top' column
df['total_top'] = df['top'].apply(convert_to_seconds)
df['opp_total_top'] = df['top_opp'].apply(convert_to_seconds)
df.head()


,season,team,week,day,date,result,ot,host,opp,pts,...,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_seconds,total_top,opp_total_top
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,1710,1710,1890
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,1880,1880,1720
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers,1412,1412,2188
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,Arizona Cardinals,1670,1670,1930
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns,Arizona Cardinals,Cleveland Browns,1863,1863,1737


In [26]:
# Create the home_results column to show if home teams won or lost 
df['home_results'] = ((df['result'] == 'W') & (df['host'] == 'home')) | ((df['result'] == 'L') & (df['host'] == 'away'))
df['home_results'] = df['home_results'].astype(int)

In [52]:
df.sample(5)

,season,team,week,day,date,result,ot,host,opp,pts,...,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results
2983,2011,JAX,13,Mon,December 5,L,no,home,Los Angeles Chargers,14,...,0,28:17,2011_13_Jacksonville Jaguars_Los Angeles Chargers,Jacksonville Jaguars,San Diego Chargers,Jacksonville Jaguars,Los Angeles Chargers,1903,1697,0
4675,2018,CHI,13,Sun,December 2,L,yes,away,New York Giants,27,...,2,33:13,2018_13_Chicago Bears_New York Giants,Chicago Bears,New York Giants,New York Giants,Chicago Bears,2038,1993,1
3168,2012,CLE,15,Sun,December 16,L,no,home,Washington Commanders,21,...,1,36:17,2012_15_Cleveland Browns_Washington Commanders,Cleveland Browns,Washington Redskins,Cleveland Browns,Washington Commanders,1423,2177,0
4393,2017,BUF,12,Sun,November 26,W,no,away,Kansas City Chiefs,16,...,1,26:42,2017_12_Buffalo Bills_Kansas City Chiefs,Buffalo Bills,Kansas City Chiefs,Kansas City Chiefs,Buffalo Bills,1998,1602,0
3633,2014,CAR,5,Sun,October 5,W,no,home,Chicago Bears,31,...,1,32:59,2014_5_Carolina Panthers_Chicago Bears,Carolina Panthers,Chicago Bears,Carolina Panthers,Chicago Bears,1621,1979,1


In [50]:
df['opp_full_name'].sort_values().unique().tolist()

['Atlanta Falcons',
 'Baltimore Ravens',
 'Buffalo Bills',
 'Carolina Panthers',
 'Chicago Bears',
 'Cincinnati Bengals',
 'Cleveland Browns',
 'Dallas Cowboys',
 'Denver Broncos',
 'Detroit Lions',
 'Green Bay Packers',
 'Houston Texans',
 'Indianapolis Colts',
 'Jacksonville Jaguars',
 'Kansas City Chiefs',
 'Las Vegas Raiders',
 'Los Angeles Chargers',
 'Los Angeles Rams',
 'Miami Dolphins',
 'Minnesota Vikings',
 'New England Patriots',
 'New Orleans Saints',
 'New York Giants',
 'New York Jets',
 'Oakland Raiders',
 'Philadelphia Eagles',
 'Pittsburgh Steelers',
 'San Diego Chargers',
 'San Francisco 49ers',
 'Seattle Seahawks',
 'St. Louis Rams',
 'Tampa Bay Buccaneers',
 'Tennessee Titans',
 'Washington Commanders',
 'Washington Football Team',
 'Washington Redskins']

In [44]:
df['home_team'] = df['home_team'].map(team_mapping)

In [51]:
df.drop(columns=['total_seconds'], inplace=True)

In [53]:
pd.options.display.max_columns = None
df.sample(3)

,season,team,week,day,date,result,ot,host,opp,pts,opp_pts,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results
4735,2018,DET,13,Sun,December 2,L,no,home,Los Angeles Rams,16,30,20,34,208,1,1,4,37,7.2,5.5,58.8,74.1,26,102,3.9,0,3,3,1,1,6,286,2,12,0,0,31:44,17,33,195,1,1,2,12,6.3,5.6,51.5,67.1,29,149,5.1,2,3,3,3,3,3,151,5,11,0,0,28:16,2018_13_Detroit Lions_Los Angeles Rams,Detroit Lions,Los Angeles Rams,Detroit Lions,Los Angeles Rams,1904,1696,0
5437,2021,CAR,16,Sun,December 26,L,no,home,Tampa Bay Buccaneers,6,32,22,45,206,0,1,7,45,5.6,4.0,48.9,52.6,15,67,4.5,0,2,2,0,0,5,253,7,18,0,2,26:36,18,31,232,1,0,0,0,7.5,7.5,58.1,92.4,31,159,5.1,2,4,4,2,2,3,123,7,14,0,0,33:24,2021_16_Carolina Panthers_Tampa Bay Buccaneers,Carolina Panthers,Tampa Bay Buccaneers,Carolina Panthers,Tampa Bay Buccaneers,1596,2004,0
367,2001,DAL,12,Sun,December 2,W,no,away,Washington Commanders,20,14,7,14,122,1,1,2,8,9.3,7.6,50.0,74.1,45,215,4.8,1,2,3,2,2,3,114,6,14,0,0,34:27,17,32,196,1,1,3,20,6.8,5.6,53.1,69.3,22,81,3.7,1,0,0,2,2,6,137,3,12,2,2,25:33,2001_12_Dallas Cowboys_Washington Commanders,Dallas Cowboys,Washington Redskins,Washington Commanders,Dallas Cowboys,2067,1533,0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 71 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                6175 non-null   int64  
 1   team                  6175 non-null   object 
 2   week                  6175 non-null   int64  
 3   day                   6175 non-null   object 
 4   date                  6175 non-null   object 
 5   result                6175 non-null   object 
 6   ot                    6175 non-null   object 
 7   host                  6175 non-null   object 
 8   opp                   6175 non-null   object 
 9   pts                   6175 non-null   int64  
 10  opp_pts               6175 non-null   int64  
 11  pass_cmp              6175 non-null   int64  
 12  pass_att              6175 non-null   int64  
 13  pass_yds              6175 non-null   int64  
 14  pass_td               6175 non-null   int64  
 15  int                  

In [57]:
df['date'].max()

'September 9'

In [60]:
def parse_month_day_with_season(row):
    try:
        # Parse the date string
        date = datetime.strptime(row['date'], "%B %d")
        # Set the year from the 'season' column
        return date.replace(year=row['season'])
    except ValueError:
        return pd.NaT

# Apply the parser to the DataFrame
df['parsed_date'] = df.apply(parse_month_day_with_season, axis=1)

In [61]:
df

,season,team,week,day,date,result,ot,host,opp,pts,opp_pts,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results,parsed_date
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,21,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,28:30,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,1710,1890,1,2000-09-03
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,31,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,31:20,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,1880,1720,1,2000-09-10
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,29,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,23:32,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers,1412,2188,0,2000-09-24
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,27,24,41,239,1,0,0,0,5.8,5.8,58.5,83.3,24,126,5.3,1,2,3,2,2,4,176,3,13,2,3,27:50,22,33,215,1,0,1,5,6.7,6.3,66.7,94.9,32,130,4.1,2,2,2,3,3,6,137,7,16,0,0,32:10,2000_5_Arizona Cardinals_San Francisco 49ers,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,Arizona Cardinals,1670,1930,1,2000-10-01
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,21,17,30,169,2,0,1,2,5.7,5.5,56.7,95.0,31,146,4.7,1,3,3,2,2,3,144,2,10,1,2,31:03,16,22,136,0,0,1,2,6.3,5.9,72.7,88.4,31,104,3.4,3,0,1,3,3,7,206,5,14,0,1,28:57,2000_6_Arizona Cardinals_Cleveland Browns,Arizona Cardinals,Cleveland Browns,Arizona Cardinals,Cleveland Browns,1863,1737,1,2000-10-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023,SEA,14,Sun,December 10,L,no,away,San Francisco 49ers,16,28,22,31,254,2,2,4,15,8.7,7.3,71.0,90.0,20,70,3.5,0,1,1,1,1,6,311,2,11,0,1,30:45,19,27,354,2,1,3,14,13.6,11.8,70.4,122.1,23,173,7.5,2,0,0,4,4,5,246,6,11,0,0,29:15,2023_14_San Francisco 49ers_Seattle Seahawks,Seattle Seahawks,San Francisco 49ers,San Francisco 49ers,Seattle Seahawks,1845,1755,1,2023-12-10
6171,2023,SEA,16,Sun,December 24,W,no,away,Tennessee Titans,20,17,25,36,215,2,0,3,12,6.3,5.5,69.4,103.4,20,58,2.9,0,2,2,2,2,3,134,7,13,0,0,26:09,19,27,125,1,0,6,39,6.1,3.8,70.4,92.4,31,162,5.2,1,1,1,2,2,3,117,6,12,1,1,33:51,2023_16_Seattle Seahawks_Tennessee Titans,Seattle Seahawks,Tennessee Titans,Tennessee Titans,Seattle Seahawks,1569,2031,0,2023-12-24
6172,2023,SFO,11,Sun,November 19,W,no,home,Tampa Bay Buccaneers,27,14,21,25,305,3,0,4,28,13.3,10.5,84.0,157.1,30,115,3.8,0,2,2,3,3,3,153,4,10,0,1,30:51,29,45,221,1,1,4,25,5.5,4.5,64.4,74.4,18,66,3.7,1,0,0,2,2,3,144,7,14,1,3,29:09,2023_11_San Francisco 49ers_Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers,1851,1749,1,2023-11-19
6173,2023,SFO,17,Sun,December 31,W,no,away,Washington Commanders,27,10,22,28,224,2,0,1,6,8.2,7.7,78.6,123.8,39,184,4.7,1,2,2,3,3,2,76,4,9,0,1,38:13,17,28,163,1,2,1,

In [63]:
df.to_csv('wip_data.csv', index=False)

In [2]:
df = pd.read_csv('wip_data.csv')
df.head(3)

,season,team,week,day,date,result,ot,host,opp,pts,...,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results,parsed_date
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,31:30,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,1710,1890,1,2000-09-03
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,28:40,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,1880,1720,1,2000-09-10
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,36:28,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers,1412,2188,0,2000-09-24


In [13]:
df['parsed_date'] = pd.to_datetime(df['parsed_date'])

In [16]:
df['season'] = df['parsed_date'].dt.year

In [21]:
df['day_of_week'] = df['parsed_date'].dt.dayofweek

In [25]:
df['month'] = df['parsed_date'].dt.month

In [23]:
df.head(1)

,season,team,week,day,date,result,ot,host,opp,pts,...,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results,parsed_date,day_of_week
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,1710,1890,1,2000-09-03,6


In [31]:
df.rename(columns={'ot':'overtime'}, inplace=True)

In [29]:
df

,season,team,week,day,date,result,ot,host,opp,pts,...,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results,parsed_date,day_of_week,month
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,1710,1890,1,2000-09-03,6,9
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,1880,1720,1,2000-09-10,6,9
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers,1412,2188,0,2000-09-24,6,9
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,Arizona Cardinals,1670,1930,1,2000-10-01,6,10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,Arizona Cardinals,Cleveland Browns,Arizona Cardinals,Cleveland Browns,1863,1737,1,2000-10-08,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,2023,SEA,14,Sun,December 10,L,no,away,San Francisco 49ers,16,...,Seattle Seahawks,San Francisco 49ers,San Francisco 49ers,Seattle Seahawks,1845,1755,1,2023-12-10,6,12
6171,2023,SEA,16,Sun,December 24,W,no,away,Tennessee Titans,20,...,Seattle Seahawks,Tennessee Titans,Tennessee Titans,Seattle Seahawks,1569,2031,0,2023-12-24,6,12
6172,2023,SFO,11,Sun,November 19,W,no,home,Tampa Bay Buccaneers,27,...,San Francisco 49ers,Tampa Bay Buccaneers,San Francisco 49ers,Tampa Bay Buccaneers,1851,1749,1,2023-11-19,6,11
6173,2023,SFO,17,Sun,December 31,W,no,away,Washington Commanders,27,...,San Francisco 49ers,Washington Commanders,Washington Commanders,San Francisco 49ers,2293,1307,0,2023-12-31,6,12


In [35]:
df['ot'] = df['overtime'].apply(lambda x: 1 if x == 'yes' else 0)

In [78]:
pd.options.display.max_columns = None
df[df['overtime'] == 'yes']

,season,team,week,day,date,result,overtime,host,opp,pts,opp_pts,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp,game_id,team_full_name,opp_full_name,home_team,away_team,total_top,opp_total_top,home_results,parsed_date,day_of_week,month,ot
52,2000,BUF,7,Sun,October 15,W,yes,home,Los Angeles Chargers,27,24,31,50,329,1,1,4,22,7.0,6.1,62.0,79.5,28,95,3.4,1,2,2,3,3,10,430,5,19,1,1,35:32,21,34,251,2,2,4,36,8.4,6.6,61.8,79.4,27,52,1.9,1,1,1,3,3,7,206,5,14,0,0,32:54,2000_7_Buffalo Bills_Los Angeles Chargers,Buffalo Bills,San Diego Chargers,Buffalo Bills,Los Angeles Chargers,2132,1974,1,2000-10-15,6,10,1
55,2000,BUF,10,Sun,November 5,W,yes,away,New England Patriots,16,13,18,37,174,1,0,1,5,4.8,4.6,48.6,71.2,31,141,4.5,0,3,3,1,1,8,288,5,15,0,0,31:54,14,27,100,0,2,3,16,4.3,3.3,51.9,29.9,34,89,2.6,1,2,2,1,1,8,269,6,17,1,1,32:38,2000_10_Buffalo Bills_New England Patriots,Buffalo Bills,New England Patriots,New England Patriots,Buffalo Bills,1914,1958,0,2000-11-05,6,11,1
61,2000,BUF,16,Sun,December 17,L,yes,home,New England Patriots,10,13,16,28,169,1,0,4,28,7.0,5.3,57.1,86.8,40,178,4.5,0,1,3,1,1,7,203,6,17,0,0,35:03,13,26,142,0,0,1,14,6.0,5.3,50.0,66.5,48,189,3.9,1,2,3,1,1,7,206,5,17,0,3,39:38,2000_16_Buffalo Bills_New England Patriots,Buffalo Bills,New England Patriots,Buffalo Bills,New England Patriots,2103,2378,0,2000-12-17,6,12,1
65,2000,CAR,5,Sun,October 1,L,yes,home,Dallas Cowboys,13,16,17,29,153,0,0,1,8,5.6,5.1,58.6,72.9,29,119,4.1,1,2,2,1,1,3,120,6,14,2,4,27:49,15,23,118,0,2,2,13,5.7,4.7,65.2,41.6,38,173,4.6,1,2,3,0,0,4,70,7,14,0,1,36:03,2000_5_Carolina Panthers_Dallas Cowboys,Carolina Panthers,Dallas Cowboys,Carolina Panthers,Dallas Cowboys,1669,2163,0,2000-10-01,6,10,1
120,2000,DAL,9,Sun,October 29,L,yes,home,Jacksonville Jaguars,17,23,17,25,207,1,1,2,13,8.8,7.7,68.0,89.9,32,118,3.7,1,1,1,2,2,3,123,5,9,0,0,30:08,20,24,231,3,0,0,0,9.6,9.6,83.3,146.4,38,161,4.2,0,1,2,2,2,5,110,7,15,0,1,33:36,2000_9_Dallas Cowboys_Jacksonville Jaguars,Dallas Cowboys,Jacksonville Jaguars,Dallas Cowboys,Jacksonville Jaguars,1808,2016,0,2000-10-29,6,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,2023,CLT,4,Sun,October 1,L,yes,home,Los Angeles Rams,23,29,11,25,196,2,0,2,4,8.0,7.3,44.0,98.1,31,133,4.3,1,0,1,1,1,4,193,3,10,1,2,24:25,27,40,303,1,1,2,16,8.0,7.2,67.5,87.8,36,164,4.6,2,3,5,2,2,3,123,6,15,1,1,39:46,2023_4_Indianapolis Colts_Los Angeles Rams,Indianapolis Colts,Los Angeles Rams,Indianapolis Colts,Los Angeles Rams,1465,2386,0,2023-10-01,6,10,1
6087,2023,CLT,13,Sun,December 3,W,yes,away,Tennessee Titans,31,28,26,42,300,2,0,3,12,7.4,6.7,61.9,99.3,23,55,2.4,0,4,4,1,1,5,256,3,14,1,1,29:03,16,33,204,1,0,6,20,6.8,5.2,48.5,78.3,42,177,4.2,2,2,2,2,3,7,299,6,17,0,1,38:26,2023_13_Indianapolis Colts_Tennessee Titans,Indianapolis Colts,Tennessee Titans,Tennessee Titans,Indianapolis Colts,1743,2306,0,2023-12-03,6,12,1
6107,2023,SDG,2,Sun,September 17,L,yes,away,Tennessee Titans,24,27,27,41,281,2,0,3,24,7.4,6.4,65.9,101.8,21,61,2.9,0,3,3,1,1,6,234,2,14,3,3,28:09,20,24,200,1,0,5,46,10.3,6.9,83.3,115.3,34,141,4.1,2,2,2,3,3,5,306,6,13,0,1,36:49,2023_2_Los Angeles Chargers_Tennessee Titans,Los Angeles Chargers,Tennessee Titans,Tennessee Titans,Los Angeles Chargers,1689,2209,1,2023-09-17,6,9,1
6154,2023,NYG,8,Sun,October 29,L,yes,home,New York Jets,10,13,6,14

In [94]:
df.columns[9:]

Index(['pts', 'opp_pts', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int',
       'sk', 'yds_lost_sks', 'yds_pass_att', 'net_yds_pass_att', 'cmp%',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 'rush_td', 'fgm',
       'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', '4dconv',
       '4datt', 'top', 'pass_cmp_opp', 'pass_att_opp', 'pass_yds_opp',
       'pass_td_opp', 'int_opp', 'sk_opp', 'yds_lost_sks_opp',
       'yds_pass_att_opp', 'net_yds_pass_att_opp', 'cmp%_opp',
       'pass_rating_opp', 'rush_att_opp', 'rush_yds_opp', 'rush_yds_att_opp',
       'rush_td_opp', 'fgm_opp', 'fga_opp', 'xpm_opp', 'xpa_opp', 'pnt_opp',
       'punt_yds_opp', '3dconv_opp', '3datt_opp', '4dconv_opp', '4datt_opp',
       'top_opp', 'game_id', 'team_full_name', 'opp_full_name', 'home_team',
       'away_team', 'total_top', 'opp_total_top', 'home_results',
       'parsed_date', 'day_of_week', 'month', 'ot'],
      dtype='object')

In [102]:
dff = df.iloc[:, [63,64,65,66,67,1,71,0,73,2,72,9,10,74] + list(range(11, 36)) + [68] + list(range(37,62)) +[69,70]] 
dff.shape

(6175, 67)

In [104]:
same_columns = set(df.columns) == set(dff.columns)

In [104]:
same_columns = set(df.columns) == set(dff.columns)

In [104]:
same_columns = set(df.columns) == set(dff.columns)

In [64]:
#opp_full_name change to og name during the era, better yet change the game_id to have the og names in it 
df[['team_full_name','opp_full_name','home_team','away_team']].nunique()

team_full_name    31
opp_full_name     36
home_team         32
away_team         32
dtype: int64

In [74]:
df['day'].value_counts()

day
Sun    5335
Mon     415
Thu     284
Sat     129
Fri       5
Tue       5
Wed       2
Name: count, dtype: int64

In [106]:
columns_only_in_df = set(df.columns) - set(dff.columns)
columns_only_in_dff = set(dff.columns) - set(df.columns)

print("Columns only in df:", columns_only_in_df)
print("Columns only in dff:", columns_only_in_dff)

Columns only in df: {'overtime', 'opp', 'result', 'host', 'date', 'top', 'top_opp', 'day'}
Columns only in dff: set()


In [110]:
dff.head(3)

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,day_of_week,pts,opp_pts,ot,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,total_top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
0,2000_1_Arizona Cardinals_New York Giants,Arizona Cardinals,New York Giants,New York Giants,Arizona Cardinals,CRD,2000-09-03,2000,9,1,6,16,21,0,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,1710,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,1890,1
1,2000_2_Arizona Cardinals_Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,Arizona Cardinals,Dallas Cowboys,CRD,2000-09-10,2000,9,2,6,32,31,0,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,1880,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,1720,1
2,2000_4_Arizona Cardinals_Green Bay Packers,Arizona Cardinals,Green Bay Packers,Arizona Cardinals,Green Bay Packers,CRD,2000-09-24,2000,9,4,6,3,29,0,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,1412,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,2188,0


In [109]:
dff.to_csv('reordered_wip_df.csv', index = False)

In [ ]:
# redo game_id with home team first then away team with og names
# write down what encoded numbers mean for legend (i.e. 0 in OT means 'no' overtime, 1 'win" in home result)
# get stats to be away vs home stats instead of opp / team stats

In [ ]:
I want to rewrite the game_id data to be the year_week_home_team_away_team. But I also want the original name of the team during that era. 

game_id	team_full_name	opp_full_name	home_team	away_team	team	parsed_date	season	month	week	day_of_week	pts	opp_pts	ot	pass_cmp	pass_att	pass_yds	pass_td	int	sk	yds_lost_sks	yds_pass_att	net_yds_pass_att	cmp%	pass_rating	rush_att	rush_yds	rush_yds_att	rush_td	fgm	fga	xpm	xpa	pnt	punt_yds	3dconv	3datt	4dconv	4datt	total_top	pass_cmp_opp	pass_att_opp	pass_yds_opp	pass_td_opp	int_opp	sk_opp	yds_lost_sks_opp	yds_pass_att_opp	net_yds_pass_att_opp	cmp%_opp	pass_rating_opp	rush_att_opp	rush_yds_opp	rush_yds_att_opp	rush_td_opp	fgm_opp	fga_opp	xpm_opp	xpa_opp	pnt_opp	punt_yds_opp	3dconv_opp	3datt_opp	4dconv_opp	4datt_opp	opp_total_top	home_results
0	2000_1_Arizona Cardinals_New York Giants	Arizona Cardinals	New York Giants	New York Giants	Arizona Cardinals	CRD	2000-09-03	2000	9	1	6	16	21	0	28	49	312	2	3	1	6	6.5	6.2	57.1	64.3	20	43	2.2	0	1	1	1	1	5	210	6	15	0	2	1710	17	25	172	0	1	0	0	6.9	6.9	68.0	70.7	41	223	5.4	3	0	0	3	3	6	155	7	14	0	0	1890	1
1	2000_2_Arizona Cardinals_Dallas Cowboys	Arizona Cardinals	Dallas Cowboys	Arizona Cardinals	Dallas Cowboys	CRD	2000-09-10	

In [131]:
df[df['parsed_date'] == pd.Timestamp('2016-01-01')]

,game_id,team_full_name,opp_full_name,home_team,away_team,team,parsed_date,season,month,week,day_of_week,pts,opp_pts,ot,pass_cmp,pass_att,pass_yds,pass_td,int,sk,yds_lost_sks,yds_pass_att,net_yds_pass_att,cmp%,pass_rating,rush_att,rush_yds,rush_yds_att,rush_td,fgm,fga,xpm,xpa,pnt,punt_yds,3dconv,3datt,4dconv,4datt,total_top,pass_cmp_opp,pass_att_opp,pass_yds_opp,pass_td_opp,int_opp,sk_opp,yds_lost_sks_opp,yds_pass_att_opp,net_yds_pass_att_opp,cmp%_opp,pass_rating_opp,rush_att_opp,rush_yds_opp,rush_yds_att_opp,rush_td_opp,fgm_opp,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,opp_total_top,home_results
4095,2016_17_Arizona Cardinals_Los Angeles Rams,Arizona Cardinals,Los Angeles Rams,Los Angeles Rams,Arizona Cardinals,CRD,2016-01-01,2016,1,17,4,44,6,0,22,42,260,3,2,1,3,6.3,6.0,52.4,75.5,24,84,3.5,1,3,3,5,5,4,147,4,14,1,2,1678,17,28,71,0,2,7,72,5.1,2.0,60.7,35.4,22,52,2.4,0,2,2,0,0,7,337,2,14,1,2,1922,0
4110,2016_17_Atlanta Falcons_New Orleans Saints,Atlanta Falcons,New Orleans Saints,Atlanta Falcons,New Orleans Saints,ATL,2016-01-01,2016,1,17,4,38,32,0,27,36,323,4,0,1,8,9.2,8.7,75.0,139.0,22,142,6.5,1,1,1,5,5,4,185,6,11,0,0,1785,29,50,341,2,1,2,9,7.0,6.6,58.0,83.8,23,132,5.7,2,2,2,2,2,4,224,7,16,3,3,1815,1
4126,2016_17_Baltimore Ravens_Cincinnati Bengals,Baltimore Ravens,Cincinnati Bengals,Cincinnati Bengals,Baltimore Ravens,RAV,2016-01-01,2016,1,17,4,10,27,0,33,51,263,0,2,3,16,5.5,4.9,64.7,61.2,16,72,4.5,1,1,1,1,1,4,175,8,16,1,3,1758,18,28,218,1,0,1,8,8.1,7.5,64.3,100.0,38,153,4.0,2,2,2,3,3,5,247,1,8,0,0,1842,1
4140,2016_17_Buffalo Bills_New York Jets,Buffalo Bills,New York Jets,New York Jets,Buffalo Bills,BUF,2016-01-01,2016,1,17,4,10,30,0,15,31,162,0,1,3,20,5.9,4.8,48.4,50.7,28,68,2.4,1,1,1,1,1,6,254,6,16,0,1,1713,20,30,202,2,0,1,8,7.0,6.5,66.7,107.9,30,127,4.2,0,3,3,3,3,5,231,4,13,0,2,1887,1
4153,2016_17_Carolina Panthers_Tampa Bay Buccaneers,Carolina Panthers,Tampa Bay Buccaneers,Tampa Bay Buccaneers,Carolina Panthers,CAR,2016-01-01,2016,1,17,4,16,17,0,18,32,207,1,3,3,30,7.4,5.9,56.3,47.3,32,128,4.0,1,1,4,1,1,4,164,4,15,2,2,1944,20,35,188,1,1,3,14,5.8,4.9,57.1,69.7,25,112,4.5,0,1,3,2,2,5,229,4,12,0,0,1656,1
4169,2016_17_Chicago Bears_Minnesota Vikings,Chicago Bears,Minnesota Vikings,Minnesota Vikings,Chicago Bears,CHI,2016-01-01,2016,1,17,4,10,38,0,13,20,140,1,2,2,9,7.5,6.4,65.0,62.5,37,183,4.9,0,1,1,1,1,3,143,2,9,1,2,1788,25,34,250,3,1,0,0,7.4,7.4,73.5,111.2,28,124,4.4,1,1,1,5,5,2,64,6,12,0,1,1812,1
4193,2016_17_Cleveland Browns_Pittsburgh Steelers,Cleveland Browns,Pittsburgh Steelers,Pittsburgh Steelers,Cleveland Browns,CLE,2016-01-01,2016,1,17,4,24,27,1,29,40,206,2,1,4,26,5.8,4.7,72.5,90.2,33,231,7.0,1,1,2,3,3,4,190,8,15,0,0,2334,24,37,243,3,1,4,34,7.5,5.9,64.9,99.3,28,69,2.5,1,0,0,3,3,8,368,3,14,2,2,1989,1
4205,2016_17_Dallas Cowboys_Philadelphia Eagles,Dallas Cowboys,Philadelphia Eagles,Philadelphia Eagles,Dallas Cowboys,DAL,2016-01-01,2016,1,17,4,13,27,0,16,29,126,1,2,3,25,5.2,3.9,55.2,48.9,21,69,3.3,0,2,2,1,1,5,195,5,12,0,0,1458,27,43,232,2,0,2,13,5.7,5.2,62.8,92.4,30,114,3.8,1,2,3,3,3,4,200,6,14,0,1,2142,1
4218,2016_17_Denver Broncos_Las Vegas Raiders,Denver Broncos,Oakland Raiders,Denver Broncos,Las Vegas Raiders,DEN,2016-01-01,2016,1,17,4,24,6,0,17,27,206,2,1,0,0,7.6,7.6,63.0,95.6,40,143,3.6,1,1,2,3,3,5,224,5,15,1,3,2125,20,32,164,1,1,2,7,5.3,4.8,62.5,72.9,16,57,3.6,0,0,0,0,0,8,452,2,11,0,1,1475,1
4231,2016_17_Detroit Lions_Green Bay Packers,Detroit Lions,Green Bay Packers,Detroit Lions,Green Bay Packers,DET,2016-01-01,2016,1,17,4,24,31,0,26,41,332,2,1,2,15,8.5,7.7,63.4,94.8,21,76,3.6,1,1,2,3,3,4,200,5,12,0,0,1551,27,39,295,4,0,1,5,7.7,7.4,69.2,125.5,31,153,4.9,0,1,1,2,3,5,186,7,13,0,0,2049,0


In [123]:
df.iloc[4334]

season                         2016
team                            SEA
week                             17
day                             Sun
date                      January 1
                       ...         
home_results                      0
parsed_date     2016-01-01 00:00:00
day_of_week                       4
month                             1
ot                                0
Name: 4334, Length: 75, dtype: object